# This is a notebook to perform LISCA analysis

Each cell corresponds to a block of code, which can be run by selecting it and pressing shift+enter. The notebook is conceived to be run sequentially. For any errors or questions contact Miguel. Make sure to select the kernel named pyama on the top right. If you cannot find a kernel named pyama, have a look at the confluence page or contact Miguel.

Run the following Cell to make all the imports

In [10]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
from nd2reader import ND2Reader
import pandas as pd
from IPython.display import display
import numpy as np
import sys
import time

sys.path.append('/project/ag-moonraedler/MAtienza/pyama/')
from lisca.pipeline import Track

from tqdm import tqdm
import os
from skimage.io import imread
from notebook_viewer import viewers
%matplotlib ipympl
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Select the folder where your imaging file is located, as well as the folder where you would like for the output of the analysis to be saved

In [2]:
data_path = '/project/ag-moonraedler/Nathalie/2022-10-11/'
nd2_file = '2022-10-11-mGL-GA890-L908.nd2'
path_out='/project/ag-moonraedler/MAtienza/pyama/forNAthalie/'

You can use this tool to have a look at your images and find out which channel (c) is BF and which which channels are fluorscence.

In [10]:
viewer = viewers.StackViewer(os.path.join(data_path, nd2_file))

GridspecLayout(children=(Canvas(header_visible=False, layout=Layout(grid_area='widget001'), toolbar=Toolbar(to…

In [12]:
plt.close(viewer.fig)

In [3]:
first=25
last=53
positions=range(first, last+1)
fl_channel=1
bf_channel=0

In [4]:
bf_channel=0
fl_channels=[1,2]

If you plan to use cellpose segmentation,use this tool for calibration by checking which parameters work best. Otherwise you may skip this part


In [17]:
segmentation_viewer = viewers.CellposeViewer(data_path+nd2_file, channel=bf_channel)

GridspecLayout(children=(Canvas(header_visible=False, layout=Layout(grid_area='widget001'), toolbar=Toolbar(to…

In [6]:
try:
    flow_threshold, diameter, mask_threshold = segmentation_viewer.flow_threshold.value, segmentation_viewer.diameter.value, segmentation_viewer.mask_threshold.value
    plt.close(segmentation_viewer.fig)
except:
    flow_threshold, diameter, mask_threshold = 0.8, 29, -2
    

## Perform LISCA pipeline

In [7]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Chose a method for segmentation from either "th", the standard pyama method or "cellpose"

In [8]:
method="th"

Choose the positions you want to evalueate

In [ ]:
%autoreload
n_positions=0
t_0 = time.time()
for fov in positions:
    print(f'Evaluating field of view {fov}')
    
    fovpath=os.path.join(path_out, f'XY{fov}/')
    if not os.path.isdir(fovpath):
        os.mkdir(fovpath)
    tracker = Track(fovpath, data_path, bf_channel, fl_channels, fov=fov, nd2_file=nd2_file)
    tracker.segment(flow_threshold=flow_threshold, mask_threshold=mask_threshold, diameter=diameter, pretrained_model='mdamb231', method=method)
    ##tracker.track(method=method)
    tracker.save_to_pyama(fl_channel)
    n_positions+=1
print(f"Time taken per position: {(time.time()-t_0)/(60*n_positions)} minutes")

Evaluating field of view 25
Running segmentation with thresholding...


100%|█████████████████████████████████████████████████████████████████| 193/193 [04:02<00:00,  1.26s/it]


Reading cyto_masks_th.mp4 ...
Done Reading
Done reading.
Done reading.
Interpolating background


 28%|██████████████████▍                                               | 54/193 [00:45<01:56,  1.19it/s]

In [ ]:
%autoreload
Resultsviewer=viewers.ResultsViewer(data_path+nd2_file, path_out)

In [96]:
Resultsviewer.df

,Unnamed: 0,id,frame,x,y,fluorescence,particle,fl_channel,particle_id
0,0,97,0,1087.177348,1415.548679,9083048704,7,eGFP_2s,7
1,1,21,0,1261.315871,245.804314,1478897803,73,eGFP_2s,73
2,2,19,0,1237.742323,237.806409,1706959216,78,eGFP_2s,78
3,3,34,0,2011.053613,460.336830,1855062480,87,eGFP_2s,87
4,4,16,0,1287.527822,197.090620,4231851809,89,eGFP_2s,89
...,...,...,...,...,...,...,...,...,...
7469,7469,7,170,1375.163693,60.363848,6246826676,93,DAPI_NS,93
7470,7470,6,170,1305.420523,75.242790,7233624178,5361,DAPI_NS,5361
7471,7471,43,170,1265.966583,569.434111,15186727935,6580,DAPI_NS,6580
7472,7472,31,170,1257.365123,338.008174,3635813412,353,DAPI_NS,353


In [2]:
from src.img_op.background_correction import background_schwarzfischer

In [ ]:
bg = background_schwarzfischer

In [8]:
1431650304*1e-9 *3

4.294950912000001

In [ ]:
np.memmap()